<img src="./images/DLI_Header.png" style="width: 400px;">

# 迁移学习

到目前为止，我们已使用大型数据集训练出了精确的模型，并且还下载了无需训练的开箱即用式预训练模型。但是，如果您找不到完全满足自身需求的预训练模型，那该怎么办？如果没有足够大的数据集可供从头训练模型，又该怎么办？本例中，我们可以使用一种非常有用的方法，此方法称为[迁移学习](https://blogs.nvidia.com/blog/2019/02/07/what-is-transfer-learning/)。

通过迁移学习，您可以利用一个预训练模型，并使用与原始训练任务存在部分重叠的任务对其进行重新训练。对于这种情况，一个绝妙的类比是，精通一种绘画方法（例如油画）的美术家想学着练习另一种绘画方法（例如素描）。可以想象，他们在创作油画时学到的技巧对于学习素描非常有用。

举个深度学习的例子，假设您有一个非常擅长识别不同汽车类型的预训练模型，现在您希望训练一个模型来识别各种类型的摩托车。那个识别汽车的模型所学到的大量知识对您可能很有帮助，比如它已经能够识别出前灯和车轮。

在您没有种类繁多的大型数据集时，迁移学习尤其有用。本例中，从头开始训练的模型很可能会快速记住训练集的数据，但却无法有效地泛化至新数据。通过迁移学习，您有更大的几率在小数据集上训练出精确、稳健的模型。

## 目标

完成本节内容的学习后，您将能够：
* 准备预训练模型以进行迁移学习
* 在预训练模型上，利用您自己的小型数据集进行迁移学习
* 进一步优化该模型，从而获得更出色的性能

## 个性化的狗门

在上一个练习中，我们使用预训练的 [ImageNet](http://www.image-net.org/) 模型允许所有狗进入，但阻止其他动物进入。本练习中，我们要创建一个只允许特定狗进入的狗门。本例中，我们将为名为 Bo 的狗制作自动狗门。您可能知道 Bo 是巴拉克·奥巴马总统家的狗。您可在 `presidential_doggy_door` 文件夹中找到 Bo 的图片。

<img src="presidential_doggy_door/train/bo/bo_10.jpg">

本练习的挑战在于，预训练模型并非专用于识别这只狗，而且我们只有 30 张 Bo 的图片。如果我们尝试使用这 30 张图片从头训练模型，则会遇到过拟合和泛化性差的问题。但是，如果我们从一个擅长检测狗的预训练模型开始，则在使用这个小型数据集的情况下，我们可利用该模型学习到的知识获得对 Bo 的一般性的了解，进而使用迁移学习解决此难题。

## 下载预训练模型

ImageNet 预训练模型是用于计算机视觉领域里的迁移学习的明智选择，因为这些模型已经学会对各种不同类型的图像进行分类。通过训练，它们学会了检测许多不同类型的 [特征](https://developers.google.com/machine-learning/glossary#)，而这些特征对于图像识别非常重要。由于 ImageNet 模型已学会检测包括狗在内的多种动物，因此它尤其适合完成检测 Bo 的迁移学习的任务。

我们先来下载预训练模型。同样地，我们可以直接从 Keras 库中获取此模型。这次下载与上一次相比将有一个重要的区别。ImageNet 模型的最后一层是包含 1000 个单元的 [密集连接层](https://developers.google.com/machine-learning/glossary#dense-layer)，用来表达数据集中的 1000 个可能的分类。本示例中，我们希望模型作出一个不同的分类：这是或不是 Bo？由于分类数量不同，因此我们将除去预训练模型的最后一层。我们可以在下载该模型时通过设置标记 `include_top=False` 来执行此操作。移除顶层后，我们可以添加新层来生成我们想要的分类类型：

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.summary()

## 冻结基础模型

在将新层添加到[预训练模型](https://developers.google.com/machine-learning/glossary#pre-trained-model)之前，我们应当执行一个重要步骤：冻结模型的预训练基础层。这意味着，在进行新的训练时，我们不会更新预训练模型的基础层。相反，我们将仅更新添加到模型末尾的新层。我们冻结基础层，是因为我们希望保留通过 ImageNet 数据集训练时所获得的学习成果。如果在此阶段未冻结这些层，我们很可能会破坏这些重要信息。将来我们可能会需要取消冻结并训练这些层，这将通过后面介绍的名为微调的过程来实现。

冻结基础层非常简单，只需将模型的`trainable` 参数设置为 `False` 即可。

In [ ]:
base_model.trainable = False

## 添加新层

现在，我们可以在预训练模型上添加新的可训练层，它们将利用预训练层中的特征并将其转变成对新数据集的预测。我们会向模型添加两个层。第一个层是池化层，这和我们先前在 [卷积神经网络](https://developers.google.com/machine-learning/glossary#convolutional_layer) 中看到的一样。（如果您想更深入地了解 CNN 中池化层的作用，请阅读 [这篇详细的博文](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~:text=A%20pooling%20layer%20is%20a,Convolutional%20Layer)）。然后，我们需要添加最终的密集连接层作为输出层，用于对 Bo 或非 Bo 图像进行分类。

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to ase 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

现在我们已将预训练模型与新层相结合，下面我们就来看一下模型。

In [ ]:
model.summary()

这里，Keras 为我们展现了一个简明扼要的结果，因为它将 vgg16 预训练模型显示为一个单元，而不是显示其所有内部层。另外，值得注意的是，由于我们已冻结预训练模型，因此会有很多不可训练的参数。

## 编译模型

和先前的练习一样，我们需要设置损失函数和评价函数选项来编译模型。但此时，我们必须选择一些不同的选项。先前的示例中，我们的分类问题中有很多类别。因此，我们选择使用多分类交叉熵 (categorical cross-entropy) 来计算损失。而本例只是一个二分类问题（Bo 或非 Bo），因此我们将使用 [二分类交叉熵 (binary cross-entropy)](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy)。如欲详细了解这两者之间的区别，请在 [此处](https://gombru.github.io/2018/05/23/cross_entropy_loss/) 获取更多内容。另外，我们还将使用二分类精度代替多分类精度作为评价函数。

通过设置 `from_logits=True`，我们可以告知 [损失函数](https://gombru.github.io/2018/05/23/cross_entropy_loss/) 输出值并未归一化（例如使用 softmax）。

In [ ]:
# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

与以前一样，这里的模型编译使用了默认优化器RMSProp。

## 增加数据

由于我们处理的数据集非常小，因此增加数据尤为重要。与先前一样，我们将对现有图像稍作修改，以便模型能够看到更多种可学习的图像。这有助于模型学习识别新的 Bo 图片，而不是只记住训练时使用过的图片。

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## 加载数据

目前为止，我们已经看到几种不同格式的数据集。在 MNIST 练习中，我们能够直接从 Keras 库中下载数据集。手语数据集的数据则存储在 CSV 文件中。对于本练习，我们将使用 Keras 的 [`flow_from_directory`](https://keras.io/api/preprocessing/image/) 函数直接从文件夹中加载图像。为了能顺利执行此操作，我们已设置好目录`presidential_doggy_door`，该目录中又分有训练和测试目录，训练和测试目录中又都包含 Bo 图像和not_Bo 图像文件夹。not_bo 目录中包含其他狗和猫的图片，用于教模型阻止其他宠物进入。您可随时浏览这些图像，加深了解我们的数据集。

请注意，[flow_from_directory](https://keras.io/api/preprocessing/image/) 函数还允许我们将图像调整为 244x244 像素并具有 3 个通道的图像，以与模型相匹配。

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('presidential_doggy_door/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate test dataset
test_it = datagen.flow_from_directory('presidential_doggy_door/test/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

## 训练模型

现在，我们就该训练模型了，看看它表现得如何。 回想一下，使用数据生成器时，我们必须显式地设置`steps_per_epoch`：

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=test_it, validation_steps=4, epochs=20)

## 讨论结果

希望您会发现训练准确率和验证准确率都非常高。这个结果很棒！能够使用小型数据集进行训练并非难事，但由于利用了从 ImageNet 模型迁移过来的知识，我们还能实现模型的高准确率和有效泛化。也就是说，我们的模型能够非常有效地区分 Bo 和不是 Bo 的宠物。

如果看到验证准确率出现一些波动，这也是正常的。我们将在下一节介绍用于改进模型的方法。

## 模型的微调

模型的新层经过训练后，我们还有一个可选的技巧，用于进一步改进模型。这个技巧称为 [微调](https://developers.google.com/machine-learning/glossary#f)。要执行此操作，我们需要先解冻整个模型，然后以极小的 [学习率](https://developers.google.com/machine-learning/glossary#learning-rate) 重新训练模型。这会让预训练的基础层以非常小的步伐进行微调，从而进一步改进模型的性能。

请注意，只有在包含冻结层和新层的模型经过充分训练后才能执行此步骤，这一点很重要。否则，由于我们先前添加到模型中的新的池化层和分类层是随机地初始化的，我们需要对它们的参数进行大量的更新才能实现准确的图像分类。而通过 [反向传播过程](https://developers.google.com/machine-learning/glossary#backpropagation)，在整个模型解冻的状态下，这种对最后两层的大量更新也可能导致预训练层中出现大量更新，从而破坏那些重要的预训练特征。现在这最后2层已经过训练且已收敛，因此再次训练时对模型整体的更新都要小得多（尤其是学习率非常小时），不会破坏前面那些层所预学到的特征。

下面我们就来尝试解冻预训练层，然后微调整个模型：

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=test_it, validation_steps=4, epochs=10)

## 检查预测结果

现在我们已对模型进行了有效训练，下面我们就来为 Bo 创建狗门！首先让我们看一下模型作出的预测。我们会按照为上一个狗门进行图像预处理的方式来预处理这个图像。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

我们在几个图像上尝试一下，并查看预测结果：

In [ ]:
make_predictions('presidential_doggy_door/test/bo/bo_20.jpg')

In [ ]:
make_predictions('presidential_doggy_door/test/not_bo/121.jpg')

看起来负数预测表示图像中是 Bo，而正数预测表示图像中是其他动物。所以，我们可以使用此信息让狗门只允许 Bo 进入！

## 练习：Bo 的狗门

填写下列代码以实现 Bo 的狗门：

In [ ]:
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if FIXME:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

## 答案

单击下方的“...”查看答案。

```python
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")
```

我们来试一试！

In [ ]:
presidential_doggy_door('presidential_doggy_door/test/not_bo/131.jpg')

In [ ]:
presidential_doggy_door('presidential_doggy_door/test/bo/bo_29.jpg')

## 总结

您做得很好！利用迁移学习，您仅使用了一个很小的数据集就构建出了一个高精度的模型。这可谓是一种非常强大的方法，能够决定项目是获得成功还是无法取得进展。我们希望这些方法今后能够在类似的情况下帮助您摆脱困境！

您可以从 [NVIDIA 迁移学习工具包](https://developer.nvidia.com/tlt-getting-started) 中获得大量有用的迁移学习资源。

### 清理显存
继续进行后续内容之前，请执行以下单元清理 GPU 显存。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

目前为止，这次研讨会的重点主要聚焦于图像分类。下一节中，为了更全面地介绍深度学习，我们将转而处理序列数据，并将采用不同的方法。

请继续学习下一节：[*序列数据*](./06_headline_generator.ipynb)。